## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Khatanga,RU,71.9667,102.5000,42.42,76,100,5.21,overcast clouds
1,1,Kutum,SD,14.2000,24.6667,77.07,54,88,1.54,overcast clouds
2,2,New Norfolk,AU,-42.7826,147.0587,46.92,77,100,2.48,overcast clouds
3,3,Kapaa,US,22.0752,-159.3190,82.09,76,75,14.97,broken clouds
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,67.96,73,75,11.50,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is your minimum desired temperature for your trip?'))
max_temp = float(input('What is your maximum desired temperature for your trip?'))

What is your minimum desired temperature for your trip?75
What is your maximum desired temperature for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']>= min_temp) & (city_data_df['Max Temp']<= max_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Kutum,SD,14.2000,24.6667,77.07,54,88,1.54,overcast clouds
3,3,Kapaa,US,22.0752,-159.3190,82.09,76,75,14.97,broken clouds
7,7,Washougal,US,45.5826,-122.3534,84.15,50,3,5.01,clear sky
8,8,Ixtapa,MX,20.7000,-105.2000,86.54,77,100,5.44,overcast clouds
9,9,Padang,ID,-0.9492,100.3543,77.18,81,100,2.35,moderate rain
11,11,Axim,GH,4.8699,-2.2405,75.15,91,5,6.08,clear sky
12,12,Atuona,PF,-9.8000,-139.0333,77.34,63,49,18.41,scattered clouds
13,13,San Patricio,US,28.0170,-97.5169,77.85,89,100,8.70,overcast clouds
16,16,Puerto Ayora,EC,-0.7393,-90.3518,76.96,87,94,7.00,overcast clouds
17,17,Hithadhoo,MV,-0.6000,73.0833,81.61,70,100,19.01,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                229
City                   229
Country                229
Lat                    229
Lng                    229
Max Temp               229
Humidity               229
Cloudiness             229
Wind Speed             229
Current Description    229
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kutum,SD,77.07,overcast clouds,14.2000,24.6667,
3,Kapaa,US,82.09,broken clouds,22.0752,-159.3190,
7,Washougal,US,84.15,clear sky,45.5826,-122.3534,
8,Ixtapa,MX,86.54,overcast clouds,20.7000,-105.2000,
9,Padang,ID,77.18,moderate rain,-0.9492,100.3543,
11,Axim,GH,75.15,clear sky,4.8699,-2.2405,
12,Atuona,PF,77.34,scattered clouds,-9.8000,-139.0333,
13,San Patricio,US,77.85,overcast clouds,28.0170,-97.5169,
16,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,
17,Hithadhoo,MV,81.61,overcast clouds,-0.6000,73.0833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Place API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError, KeyError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kutum,SD,77.07,overcast clouds,14.2000,24.6667,Dibbo areaمنطقة دبو
3,Kapaa,US,82.09,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
7,Washougal,US,84.15,clear sky,45.5826,-122.3534,Camas Boutique Hotel
8,Ixtapa,MX,86.54,overcast clouds,20.7000,-105.2000,hotel la terminal
9,Padang,ID,77.18,moderate rain,-0.9492,100.3543,Mercure Padang
...,...,...,...,...,...,...,...
683,Sunland Park,US,88.25,clear sky,31.7965,-106.5800,SpringHill Suites by Marriott El Paso
686,Riyadh,SA,89.74,clear sky,24.6877,46.7219,"InterContinental Riyadh, an IHG Hotel"
693,Shiraz,IR,80.04,clear sky,29.6036,52.5388,Shams Hotel
695,Misratah,LY,83.91,clear sky,32.3754,15.0925,Almasa Hotel


In [11]:
# 8a. Create the output File (CSV)

output_data_file = ('WeatherPy_vacation.csv')

# 8b. Export the City_Data into a csv
#clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map.

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))